# Assessing accuracy of TrOCR model using IAM Dataset

In this project, I sought to use a recent optical character recognition model from this paper: https://arxiv.org/pdf/2109.10282

I begin by playing with the TrOCR model with some handwritten text of my own. I determine the effectiveness of the model both without noise and in the presence of Gaussian and Poisson noise.

Next, I input text from the famous IAM Handwritten Database. I assess the accuracy of the model in identifying written words both before and after the addition of noise.

In [5]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

#

In [9]:
# load trial image from the IAM database
url = 'https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
image.show()

In [11]:
# Access Pretrained TrOCR Model
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.47.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

In [13]:

pixel_values = processor(images=image, return_tensors="pt").pixel_values

In [15]:
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [17]:
print(generated_text)

indus the
